# DB Tools 常见问法示例（教学版）

本 notebook 用“常见问法 + 说明 + 自然语言调用示例”的方式讲解订单 DB 工具的用法。
已按功能分区，适合课堂演示或新同学快速上手。


## 0. 环境准备（分步）

### Step 1: 准备依赖与环境
确保项目依赖已安装，并能在本机启动 Python。

### Step 2: 配置环境变量
在项目根目录 `.env` 中配置以下变量：
- `SUPABASE_URL`
- `SUPABASE_SERVICE_ROLE_KEY` 或 `SUPABASE_ANON_KEY`
- `OPENAI_API_KEY`

### Step 3: 载入环境变量
下面的代码会通过 `load_dotenv()` 自动读取 `.env`。

### Step 4: 理解调用方式
本示例用 `app.invoke` 让 LLM 通过自然语言自动选择并调用工具。
你只需要准备好自然语言输入，其余由工具编排完成。


In [9]:
from __future__ import annotations

from dataclasses import asdict
from datetime import datetime
from typing import Any, Dict

from dotenv import load_dotenv

# LangChain / LangGraph
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition

# 业务层：Supabase 操作封装
from services.order_services import (
    create_db_client,
    add_order_to_db,
    cancel_order,
    deliver_order,
    edit_order_from_db,
    finish_order,
    get_order_detail,
    mark_order_paid,
    order_to_text,
)
from services.types import (
    ConflictError,
    ConstraintError,
    NotFoundError,
    TerminalOrderError,
    ValidationError,
)

load_dotenv()


True

In [10]:
def _order_to_dict(order) -> Dict[str, Any]:
    return asdict(order)

@tool
# Keyword-only args keep tool schema explicit for the LLM.
def create_order_tool(
    *,
    order_id: str,
    user_name: str,
    user_wechat: str,
    sku: str,
    start_at: str,
    end_at: str,
    status: str | None = None,
    buffer_hours: int | None = None,
    locker_code: str | None = None,
) -> Dict[str, Any]:
    """Create an order."""
    try:
        client = create_db_client()
        data = {
            "order_id": order_id,
            "user_name": user_name,
            "user_wechat": user_wechat,
            "sku": sku,
            "start_at": datetime.fromisoformat(start_at),
            "end_at": datetime.fromisoformat(end_at),
        }
        if status is not None:
            data["status"] = status
        if buffer_hours is not None:
            data["buffer_hours"] = buffer_hours
        if locker_code is not None:
            data["locker_code"] = locker_code
        order = add_order_to_db(client=client, **data)
        return {"result": _order_to_dict(order)}
    except (ConflictError, ConstraintError, ValidationError, ValueError) as exc:
        return {"error": f"{exc.__class__.__name__}: {exc}"}

@tool
# Keyword-only args keep tool schema explicit for the LLM.
def get_order_tool(*, order_id: str) -> Dict[str, Any]:
    """Get order detail."""
    try:
        client = create_db_client()
        order = get_order_detail(order_id, client=client)
        return {"result": order_to_text(order)}
    except NotFoundError as exc:
        return {"error": f"{exc.__class__.__name__}: {exc}"}

@tool
# Keyword-only args keep tool schema explicit for the LLM.
def update_order_tool(*, order_id: str, patch: Dict[str, Any]) -> Dict[str, Any]:
    """Update order by patch."""
    try:
        client = create_db_client()
        order = edit_order_from_db(order_id, patch=patch, client=client)
        return {"result": _order_to_dict(order)}
    except (ConflictError, ConstraintError, ValidationError, NotFoundError, TerminalOrderError) as exc:
        return {"error": f"{exc.__class__.__name__}: {exc}"}

@tool
# Keyword-only args keep tool schema explicit for the LLM.
def cancel_order_tool(*, order_id: str, hard_delete: bool = False) -> Dict[str, Any]:
    """Cancel order."""
    try:
        client = create_db_client()
        order = cancel_order(order_id, hard_delete=hard_delete, client=client)
        return {"result": _order_to_dict(order)}
    except (NotFoundError, ValidationError, TerminalOrderError) as exc:
        return {"error": f"{exc.__class__.__name__}: {exc}"}

@tool
# Keyword-only args keep tool schema explicit for the LLM.
def mark_paid_tool(*, order_id: str) -> Dict[str, Any]:
    """Mark order paid."""
    try:
        client = create_db_client()
        order = mark_order_paid(order_id, client=client)
        return {"result": _order_to_dict(order)}
    except (NotFoundError, ValidationError, TerminalOrderError) as exc:
        return {"error": f"{exc.__class__.__name__}: {exc}"}

@tool
# Keyword-only args keep tool schema explicit for the LLM.
def deliver_order_tool(*, order_id: str, locker_code: str) -> Dict[str, Any]:
    """Deliver order with locker_code."""
    try:
        client = create_db_client()
        order = deliver_order(order_id, locker_code=locker_code, client=client)
        return {"result": _order_to_dict(order)}
    except (NotFoundError, ValidationError, ConstraintError, TerminalOrderError) as exc:
        return {"error": f"{exc.__class__.__name__}: {exc}"}

@tool
# Keyword-only args keep tool schema explicit for the LLM.
def finish_order_tool(*, order_id: str) -> Dict[str, Any]:
    """Finish order."""
    try:
        client = create_db_client()
        order = finish_order(order_id, client=client)
        return {"result": _order_to_dict(order)}
    except (NotFoundError, ValidationError, TerminalOrderError) as exc:
        return {"error": f"{exc.__class__.__name__}: {exc}"}


In [11]:
TOOLS = [
    create_order_tool,
    get_order_tool,
    update_order_tool,
    cancel_order_tool,
    mark_paid_tool,
    deliver_order_tool,
    finish_order_tool,
]

llm = init_chat_model(
    model="gpt-5-nano",
    temperature=0,
).bind_tools(TOOLS)

def agent_node(state: MessagesState) -> Dict[str, Any]:
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

tool_node = ToolNode(TOOLS)

graph = StateGraph(MessagesState)
graph.add_node("agent", agent_node)
graph.add_node("tools", tool_node)
graph.set_entry_point("agent")
graph.add_conditional_edges("agent", tools_condition, {"tools": "tools", "__end__": "__end__"})
graph.add_edge("tools", "agent")
app = graph.compile()


## 1. 订单创建（Create）

适用场景：新用户下单，或客服手动录入订单。
重点字段：`order_id`（唯一）、`start_at`/`end_at`（ISO 时间字符串）、`sku`（商品编码）。

常见问法：
- “帮我创建一个新的租赁订单。” → `create_order_tool`
- “新增订单，状态先设为 pending。” → `create_order_tool`


In [13]:
# 自然语言触发创建订单
app.invoke({
    "messages": [
        HumanMessage(content=(
            "创建订单：订单号 ORD-10001，用户张三，微信号 zhangsan_001，"
            "SKU 是 WHITE_M，开始时间 2025-01-10 10:00，"
            "结束时间 2025-01-12 18:00，状态 reserved，预留缓冲 2 小时。"
        ))
    ]
})


{'messages': [HumanMessage(content='创建订单：订单号 ORD-10001，用户张三，微信号 zhangsan_001，SKU 是 WHITE_M，开始时间 2025-01-10 10:00，结束时间 2025-01-12 18:00，状态 reserved，预留缓冲 2 小时。', additional_kwargs={}, response_metadata={}, id='aacdb658-f44f-4255-a9cf-b161814147a6'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 470, 'prompt_tokens': 389, 'total_tokens': 859, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cz46f2dJfSfDbkc4qfyXfOQsTiMs4', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bccef-d9bd-7d41-9cee-3f2a7efe77ab-0', tool_calls=[{'name': 'create_order_tool', 'args': {'order_id': 'ORD-10001', 'user_name': '张三', 'user_wecha

## 2. 订单查询（Read）

适用场景：客户询问订单详情，或运营核对订单信息。
返回值：已格式化的订单文本（便于直接展示给用户）。

常见问法：
- “查询订单 ORD-10001 的详情。” → `get_order_tool`
- “给我看下这笔订单的当前状态。” → `get_order_tool`


In [14]:
# 自然语言触发查询订单
app.invoke({
    "messages": [
        HumanMessage(content="查询订单 ORD-10001 的详情")
    ]
})

{'messages': [HumanMessage(content='查询订单 ORD-10001 的详情', additional_kwargs={}, response_metadata={}, id='37703db3-ac51-4823-ab5c-32a5b45f562b'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 156, 'prompt_tokens': 330, 'total_tokens': 486, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cz49wqDgo59mPz0VN2oWQTC6qNxrR', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bccf2-f1ea-7ec2-b9a0-f1a88c67e4c7-0', tool_calls=[{'name': 'get_order_tool', 'args': {'order_id': 'ORD-10001'}, 'id': 'call_SmqEUDMGWzW0D2uZWL3AiNEA', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 330, 'output_token

TypeError: 'CompiledStateGraph' object is not subscriptable

## 3. 订单修改（Update）

适用场景：用户改期、修改时段或补充字段信息。
入参是 `patch` 字典，字段名与订单字段一致。

常见问法：
- “把订单的结束时间改成明天下午 6 点。” → `update_order_tool`
- “更新订单的备注/字段。” → `update_order_tool`


In [17]:
# 自然语言触发更新订单
app.invoke({
    "messages": [
        HumanMessage(content=(
            "把订单 ORD-10001 的结束时间改为 2025-01-15 18:00"
        ))
    ]
})


{'messages': [HumanMessage(content='把订单 ORD-10001 的结束时间改为 2025-01-15 18:00', additional_kwargs={}, response_metadata={}, id='3d0d4a38-6ba0-49c7-b201-368e025f3047'),
  AIMessage(content='我可以帮你把 ORD-10001 的结束时间改为 2025-01-15 18:00。请确认两点以便我正确执行：\n\n- 时区：请告知要使用的时区。若无特别要求，我将以系统默认时区处理（通常是 Asia/Shanghai）。\n- 是否需要同时更新其他字段（如结束日期前后的小数点、缓冲时间等）？\n\n确认后我就去执行更新。也可以先我先帮你读取一下该订单的当前信息，确认无误再更新。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1921, 'prompt_tokens': 344, 'total_tokens': 2265, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1792, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cz4DcH8iUEiUUgbBIl7L2N4iFWJyT', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bccf6-63e3-7ee2-9558-54c98f6c65ac-0', 

## 4. 订单取消（Cancel）

适用场景：用户主动取消或客服撤销订单。
`hard_delete=False` 为软取消（保留记录），`True` 会尝试删除记录。

常见问法：
- “取消这笔订单。” → `cancel_order_tool`
- “把这笔订单彻底删除。” → `cancel_order_tool`（`hard_delete=True`）


In [ ]:
# 自然语言触发取消订单
app.invoke({
    "messages": [
        HumanMessage(content="取消订单 ORD-10001")
    ]
})


## 5. 付款状态（Payment）

适用场景：确认收款后将订单标记为已付款。
该操作会触发业务状态流转（若订单已终止会报错）。

常见问法：
- “这笔订单已付款。” → `mark_paid_tool`
- “把订单状态改为已付款。” → `mark_paid_tool`


In [18]:
# 自然语言触发标记已付款
app.invoke({
    "messages": [
        HumanMessage(content="订单 ORD-10001 已付款，请更新状态")
    ]
})


{'messages': [HumanMessage(content='订单 ORD-10001 已付款，请更新状态', additional_kwargs={}, response_metadata={}, id='35874272-f73e-4cb0-be78-b3670cbb8ce2'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 348, 'prompt_tokens': 332, 'total_tokens': 680, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cz4EYDv3PJjIM7Ql9UKHi9cgMCvNR', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bccf7-4a40-72d3-a042-d0882f4f3c42-0', tool_calls=[{'name': 'mark_paid_tool', 'args': {'order_id': 'ORD-10001'}, 'id': 'call_bkm0j5Zdxkf84FLBCsBOSVtY', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 332, 'output_t

## 6. 发货交付（Delivery）

适用场景：订单已准备好并发货，需要填写柜机码或取件码。
`locker_code` 为必填字段，用于用户取件。

常见问法：
- “请发货并填入柜机码。” → `deliver_order_tool`
- “订单已发出，柜机码是 A1-09。” → `deliver_order_tool`


In [19]:
# 自然语言触发发货交付
app.invoke({
    "messages": [
        HumanMessage(content="订单 ORD-10001 已发货，柜机码是 A1-09")
    ]
})


{'messages': [HumanMessage(content='订单 ORD-10001 已发货，柜机码是 A1-09', additional_kwargs={}, response_metadata={}, id='6fed5057-c290-48a1-a35a-8578b5590e9a'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 293, 'prompt_tokens': 339, 'total_tokens': 632, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cz4F6CJ2B6bSTcWoylSQvH3TrJSKL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bccf7-d396-78d1-8f4c-522f9d82c481-0', tool_calls=[{'name': 'deliver_order_tool', 'args': {'order_id': 'ORD-10001', 'locker_code': 'A1-09'}, 'id': 'call_QnbuFjBTM1HoG7Z07XnyBWtU', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadat

## 7. 完成订单（Finish）

适用场景：用户确认归还/租期结束，订单完成闭环。
完成后通常不可再修改关键字段。

常见问法：
- “订单已完成。” → `finish_order_tool`
- “可以结束这笔订单了。” → `finish_order_tool`


In [20]:
# 自然语言触发完成订单
messages = app.invoke({
    "messages": [
        HumanMessage(content="订单 ORD-10001 已完成，请结束流程")
    ]
})

for m in messages["messages"]:
    m.pretty_print()


================================ Human Message =================================

订单 ORD-10001 已完成，请结束流程
================================== Ai Message ==================================
Tool Calls:
  finish_order_tool (call_slfwpsFUEMmYy7KNOJrmTw7w)
 Call ID: call_slfwpsFUEMmYy7KNOJrmTw7w
  Args:
    order_id: ORD-10001
================================= Tool Message =================================
Name: finish_order_tool

{'result': {'order_id': 'ORD-10001', 'user_name': '张三', 'user_wechat': 'zhangsan_001', 'sku': 'WHITE_M', 'start_at_iso': datetime.datetime(2025, 1, 10, 10, 0, tzinfo=datetime.timezone.utc), 'end_at_iso': datetime.datetime(2025, 1, 12, 18, 0, tzinfo=datetime.timezone.utc), 'buffer_hours': 2, 'status': 'successful', 'locker_code': 'A1-09', 'created_at': datetime.datetime(2026, 1, 17, 17, 10, 39, 995869, tzinfo=datetime.timezone.utc), 'updated_at': datetime.datetime(2026, 1, 17, 17, 10, 39, 995869, tzinfo=datetime.timezone.utc)}}
================================== Ai M